# [Advent of Code 2018](https://adventofcode.com/2018)
I'm going to have a go at the Advent of Code 2018 puzzles using a Jupyter notebook.
First, let's import some standard modules.

In [1]:
import collections
import functools
import itertools as its
import math
import operator
import re
import string

Also, here are some general purpose functions.

In [2]:
def input(day):
    'Load the input for `day`'
    return open(f'input/{day}').read()

def ilen(xs):
    'Return the length of an iterable'
    return sum(1 for _ in iter(xs))

def last(xs):
    'Return the last element of an iterable sequence.'
    xs = iter(xs)
    return collections.deque(xs, maxlen=1)[0]

def ints(s):
    'Return a list of integer tokens in `s`.'
    return [int(x) for x in re.findall(r'[-+]?\d+', s)]

def array(data):
    'Convert the data into a list of lists of ints.'
    return [ints(line) for line in data.splitlines()]

def swap(xs, i, j):
    'Swap items in a sequence.'
    xs[i], xs[j] = xs[j], xs[i]

def nth(xs, n):
    'Return the nth element of a sequence.'
    return next(its.islice(xs, n, None))

def take(xs, n):
    'Return (up to) first `n` items of a sequence'
    return [x for x, _ in zip(xs, range(n))]

def overlapping(xs, n):
    '''Generate overlapping n-tuples from `xs`

    overlapping('ABCDEFG', 3) --> ABC BCD CDE DEF EFG
    '''
    if isinstance(xs, collections.abc.Sequence):
        yield from (xs[i:i+n] for i in range(len(xs) + 1 - n))
    else:
        result = collections.deque(maxlen=n)
        for x in xs:
            result.append(x)
            if len(result) == n:
                yield tuple(result)
                
def pairwise(iterable):
    's -> (s0,s1), (s1,s2), (s2, s3), ...'
    return overlapping(iterable, 2)

def manhattan_distance(p, q):
    'Return the city block distance from `p` to `q`'
    return math.fabs(p[0]-q[0]) + math.fabs(p[1]-q[1])

def neighbours4(x, y):
    'Return the horizontal and vertical neighbours of `(x, y)`'
    return (x,y-1),(x-1,y),(x+1,y),(x,y+1)

def unique(xs):
    'Return `True` iff no elements of `xs` are repeated'
    return len(xs) == len(set(xs))

def mapt(f, xs):
    return tuple(map(f, xs))

flatten = its.chain.from_iterable
cat = ''.join
first = operator.itemgetter(0)
second = operator.itemgetter(1)

## [Day 1](https://adventofcode.com/2018/day/1)
The input is a series of integers, one per line. These integers represent frequency changes. Part one requires the final frequenct. That is, the sum of these integers.

In [3]:
deltas = ints(input(1)) 
sum(deltas)

599

Part two explains the frequency changes repeat, over and over. We are asked to find the first frequency which is reached a second time.

In [4]:
seen = set()
for freq in its.accumulate(its.cycle(deltas)):
    if freq in seen:
        break
    seen.add(freq)

freq

81204

## [Day 2](https://adventofcode.com/2018/day/2)
The input is a list of strings representing box ids.

In [5]:
box_ids = input(2).split()

We need to know the counts of letters in these ids -- specifically, how many ids have letters repeated 2 and/or 3 times? `Counter` can help here.

In [6]:
box_counts = [collections.Counter(box_id) for box_id in box_ids]
x2 = sum(2 in box_count.values() for box_count in box_counts)
x3 = sum(3 in box_count.values() for box_count in box_counts)

We're asked for the product of the repeat counts.

In [7]:
x2 * x3

5681

Part two asks for something quite different: to identify a pair of ids which differ by a single character.

In [8]:
len(box_ids)

250

For such a low number of ids, we simply check every pair.

In [9]:
def diffs(b, B):
    'Return the number of characters which differ in the inputs'
    return sum(c != C for c, C in zip(b, B))

diff_by_one = [p for p in its.combinations(box_ids, 2) if diffs(*p) == 1]

# Check there's just one pair which fits the criteria
assert len(diff_by_one) == 1

We're asked for the letters common to this pair.

In [10]:
''.join(c for c, C in zip(*diff_by_one[0]) if c == C)

'uqyoeizfvmbistpkgnocjtwld'

## [Day 3](https://adventofcode.com/2018/day/3)
This puzzle is about *claim*s, which are regions in a grid. Each grid square is specified by an id, then left, top, width, height values. We could write a regex to parse/check, but simply extracting `ints` gets us there.

In [11]:
claim = "#123 @ 3,2: 5x4"
ints(claim) 

[123, 3, 2, 5, 4]

For part one, we want to know how many squares on the grid are covered by more than one claim. Let's find the total area of all the claims.

In [12]:
claims = array(input(3))
sum(c[3]*c[4] for c in claims)

492476

Given this figure, we can just count the number of claims in each cell. We can then read the number of squares with 2 or more claims.

In [13]:
def rectangle(id_, L, T, W, H):
    'Generate the points within a claim'
    return its.product(range(L, L+W), range(T, T+H))

grid = collections.Counter(
    p for claim in claims for p in rectangle(*claim))

ilen(N for N in grid.values() if N > 1)

112378

For part two, we want the id of the claim which doesn't overlap any other claim. That's the claim for which cell counts are all `1`.

In [14]:
non_overlaps = [claim[0] for claim in claims
                if all(grid[p] == 1 for p in rectangle(*claim))]
assert len(non_overlaps) == 1
non_overlaps[0]

603

## [Day 4](https://adventofcode.com/2018/day/4)
It's not entirely clear what's wanted here. Are we only concerned with guards being asleep during the midnight hour? Or, when we sort the inputs, are these the only times guards are asleep? Let's start by sorting the records.

    sort < input/4 > input/4a

Note that alphabetically sorting the records places them in date order. Once sorted, my questions about the input are clarified. Every night just a single guard is on duty and that guard only sleeps during the midnight hour.

We scan through the sorted records and keep a counter for each guard. That counter records minutes in the midnight hour the guard was asleep. For example, `sleeping[42][5] == 3` would mean guard `#42` was asleep on 3 nights between `00:05` and `00:06`.

In [15]:
sleeping = collections.defaultdict(collections.Counter)

for record in open('input/4a'):
    yy,mm,dd,hh,mm,*rest = ints(record)
    if 'shift'  in record:
        guard = rest[0]
    elif 'asleep' in record:
        assert hh == 0
        asleep = mm
    else:
        assert hh == 0
        assert 'wakes' in record
        sleeping[guard].update(range(asleep, mm))

# Let's have a look at some data & check it looks OK
sleeping[2417].most_common(5)

[(39, 10), (40, 10), (41, 10), (42, 10), (36, 9)]

For part one, we're after the guard who slept the longest and the minute they were most often asleep.

In [16]:
guard, asleep = max(sleeping.items(), key=lambda i: sum(i[1].values()))
asleep.most_common(2)

[(45, 15), (44, 14)]

That's good -- we have a unique choice for the minute the guard was most often asleep. We can get the answer directly.

In [17]:
guard * asleep.most_common(1)[0][0]

119835

Part two asks: Of all guards, which guard is most frequently asleep on the same minute? The answer is in our `sleeping` dict.

In [18]:
def sleepiest_minute(guard_mins):
    guard, mins = guard_mins
    minute, freq = mins.most_common(1).pop()
    return freq, minute, guard

freq, minute, guard = max(map(sleepiest_minute, sleeping.items()))
guard * minute

12725

## [Day 5](https://adventofcode.com/2018/day/5)
Pairs of characters are repeatedly removed from a polymer -- represented as a string. First, find all possible pairs. 

In [19]:
import string
az, AZ = string.ascii_lowercase, string.ascii_uppercase
removes = [cat(p) for p in zip(az, AZ)] + [cat(p) for p in zip(AZ, az)]

Now we keep processing the polymer until there's nothing to remove.

In [20]:
def reactions(polymer, removes):
    '''Generate the series of values taken by the polymer
    
    Yield `polymer`, then the repeated result of applying all removals.
    '''
    while True:
        yield polymer
        for rm in removes:
            polymer = polymer.replace(rm, '')

def reacted(polymer, removes):
    'Keep applying removals until the polymer stabilises'
    return next(p for p in pairwise(reactions(polymer, removes))
                if len(set(p)) == 1)[0]

polymer = input(5).strip() # (strip in case a newline appeared in the file)
len(reacted(polymer, removes))

9562

For part two, we see what happens if we remove all instances of a unit before reacting. Which unit results in the biggest reaction?

In [21]:
inputs = (polymer.replace(x, '').replace(X, '') for x, X in zip(az, AZ))
min(len(reacted(p, removes)) for p in inputs)

4934

## [Day 6](https://adventofcode.com/2018/day/6)
We are filling an infinite grid, stepping in horizontal or vertical direction. 

In [22]:
coords = set(tuple(ints(row)) for row in input(6).splitlines())

# Find the X and Y ranges of the pts
x, X = min(map(first,  coords)), max(map(first,  coords))
y, Y = min(map(second, coords)), max(map(second, coords))

def on_edge(pt):
    'Return `True` if `pt` is on the bounding box of `coords`'
    return pt[0] in {x, X} or pt[1] in {y, Y}

def closest(pt, coords, dist=manhattan_distance):
    'Return the `coords` closest to `pt`'
    def key(p): return dist(p, pt)
    d, result = next(its.groupby(sorted(coords, key=key), key=key))
    return tuple(result)

# Find the closest coord to each point in the bounding box of `coords`
closest_coords = {p: closest(p, coords) 
                  for p in its.product(range(x, X+1), range(y, Y+1))}

# We are looking for the coord surrounded by the largest contained region. 

# We can eliminate any coord which is closest to a point on 
# the bounding box -- such points are not contained
infinite_extent = set(
    flatten(vs for p, vs in closest_coords.items() if on_edge(p)))

# Now count up points closest to `coords` -- this gives us areas of
# contained regions
areas = collections.Counter(
    v for v, *tied in closest_coords.values()
    if not tied and v not in infinite_extent)

coord, biggest_area = areas.most_common(1)[0]
biggest_area

3894

For part two, we want to find the region in which all points have a total Manhattan distance from `pts` of less than 10000.

In [23]:
min(sum(manhattan_distance(p, q) for q in coords) 
    for p in closest_coords if on_edge(p)) 

11299.0

Since 11299 > 10000, this check confirms the region is strictly within the bounding box of the coords.

In [24]:
ilen(p for p in its.product(range(x, X+1), range(y, Y+1))
     if sum(manhattan_distance(p, q) for q in coords) < 10000)

39398

## [Day 7](https://adventofcode.com/2018/day/7)
Part one is a topological sort, with the minor twist of ordering dependencies alphabetically when there's a choice.

In [25]:
# Each row in the input looks something like
row = 'Step Y must be finished before step J can begin.'
# We can parse it like this
select = operator.itemgetter(1, 7)
select(row.split())

('Y', 'J')

In [26]:
edges = sorted(select(row.split()) for row in input(7).splitlines())

graph = collections.defaultdict(list,
    {v: mapt(second, deps) for v, deps in its.groupby(edges, key=first)})

in_degrees = collections.Counter(map(second, edges))
ready = set(graph) - set(in_degrees)
topo = []
while ready:
    v = min(ready)
    topo.append(v)
    ready.remove(v)
    in_degrees.subtract(collections.Counter(graph[v]))
    ready |= {v for v, n in in_degrees.items() 
              if n==0 and v not in topo}
    
''.join(topo)

'CHILFNMORYKGAQXUVBZPSJWDET'

Part two adds extra workers (elves) and assigns times to the steps. We are asked for the time to complete the tasks.

In [27]:
in_degrees = collections.Counter(map(second, edges))
ready = set(graph) - set(in_degrees)
workers = 5

def t_step(S):
    'Step A takes 61 seconds, B 62 seconds etc.'
    return 61 + ord(S) - ord('A')

t = 0
work = []
topo = []

while ready or work:
    if ready and workers:
        workers -= 1
        v = min(ready)
        ready.remove(v)
        work.append((t + t_step(v), v))
    else:
        workers += 1
        work.sort(reverse=True)
        t, v = work.pop()
        topo.append(v)
        in_degrees.subtract(collections.Counter(graph[v]))
        ready |= {v for v, n in in_degrees.items() 
                  if n==0 and v not in topo and v not in mapt(second, work)}

print(t)

891


## [Day 8](https://adventofcode.com/2018/day/8)
A list of integers defines a tree structure.

In [28]:
def traverse(tree_data, i, callback):
    children, metas = tree_data[i], tree_data[i+1]
    i += 2
    for _ in range(children):
        i = traverse(tree_data, i, callback)
    callback(tree_data[i: i+metas])
    return i + metas

metadata = []

traverse(ints('2 3 0 3 10 11 12 1 1 0 1 99 2 1 1 2'), 0, metadata.extend)
assert sum(metadata) == 138

metadata.clear()
traverse(ints(input(8)), 0, metadata.extend)
sum(metadata)

44838

In [29]:
def traverse2(tree_data, i):
    children, metas = tree_data[i], tree_data[i+1]
    i += 2
    child_values = {}
    for j in range(children):
        i, v = traverse2(tree_data, i)
        child_values[j+1] = v
    
    md = tree_data[i: i+metas]
    if children:
        value = sum(child_values.get(m, 0) for m in md)
    else:
        value = sum(md)
    return i + metas, value

assert traverse2(ints('2 3 0 3 10 11 12 1 1 0 1 99 2 1 1 2'), 0)[1] == 66
traverse2(ints(input(8)), 0)[1]

22198

## [Day 9](https://adventofcode.com/2018/day/9)
Inserting numbers into a circular buffer, following some bizarre rules.

In [30]:
def play_marbles():
    marbles, pos = [0], 0
    for N in its.count(1):
        if N % 23:
            pos = (pos + 2) % len(marbles)
            marbles.insert(pos, N)
            yield 0
        else:
            pos = (pos - 7) % len(marbles)
            yield N + marbles.pop(pos)

def marbles_game(players, marbles):
    scores = collections.Counter()
    for score, player in zip(
        its.islice(play_marbles(), marbles), its.cycle(range(players))):
        scores[player] += score
    return max(scores.values())

assert marbles_game(10, 1618) == 8317
assert marbles_game(13, 7999) == 146373
assert marbles_game(17, 1104) == 2764
assert marbles_game(30, 5807) == 37305

In [31]:
# My input: 413 players; last marble is worth 71082 points
%time marbles_game(413, 71082)

Wall time: 653 ms


416424

For the second part, I wrote a [C++ program](./day9/main.cpp) using a doubly-linked list so I could insert and delete nodes in constant time. This gave the answer `3498287922`.

## [Day 10](https://adventofcode.com/2018/day/10)
Points move in a grid until they form a message.

In [32]:
pts_vs = array(input(10))

def lights(pts_vs):
    'Generate the light positions from the `pts_vs` point+velocity 4-tuples.'
    posns = [(its.count(x, vx), its.count(y, vy)) for x, y, vx, vy in pts_vs]
    while True:
        yield [(next(x), next(y)) for x, y in posns]

def bounding_box(pts):
    ''
    xs, ys = mapt(first, pts), mapt(second, pts)
    return range(min(xs), max(xs)+1), range(min(ys), max(ys)+1)

def area(box):
    w, h = box
    return len(w) * len(h)

this_pts, next_pts = its.tee(lights(pts_vs))
next(next_pts)

# We'll assume the message appears when the bounding box of 
# the points reaches a minimum -- when we print the result
# we'll know if we were right :-)
for seconds, (this, next_) in enumerate(zip(this_pts, next_pts)):
    if area(bounding_box(this)) < area(bounding_box(next_)):
        break

# Check the bounding box looks sane
bounding_box(this)

(range(141, 203), range(133, 143))

In [33]:
def message(pts):
    range_x, range_y = bounding_box(pts)
    def row(y):
        return ''.join('*' if (x,y) in pts else ' ' 
                       for x in range_x)
    print('\n'.join(row(y) for y in range_y))
    
message(this)

******  *****   *****   *****   *****   *****   ******    **  
*       *    *  *    *  *    *  *    *  *    *       *   *  * 
*       *    *  *    *  *    *  *    *  *    *       *  *    *
*       *    *  *    *  *    *  *    *  *    *      *   *    *
*****   *****   *****   *****   *****   *****      *    *    *
*       *       *  *    *    *  *  *    *  *      *     ******
*       *       *   *   *    *  *   *   *   *    *      *    *
*       *       *   *   *    *  *   *   *   *   *       *    *
*       *       *    *  *    *  *    *  *    *  *       *    *
*       *       *    *  *****   *    *  *    *  ******  *    *


Part two of the question asks how long it would take to get this answer. I went back and modified my `for` loop to enumerate the zipped values, making the answer available.

In [34]:
seconds

10027

## [Day 11](https://adventofcode.com/2018/day/11)
We are asked to find the 3x3 square in a 300x300 grid with the largest total power level. 

In [35]:
# The power level for cell (x, y) is calculated from its coords
# and a "serial number"
def hundreds(n):
    'Return the hundreds digit in the decimal representation of `n`'
    return (n//100)%10

def power_level(serial_number, x, y):
    rack_id = x + 10
    return hundreds((rack_id*y + serial_number)*rack_id) - 5

assert power_level(8, 3, 5) == 4
assert power_level(71, 101, 153) == 4

In [36]:
def power_levels(serial_number, S):
    'Calculate power levels for square side `S` using supplied `serial_number`'
    return {p: power_level(serial_number, *p) 
            for p in its.product(range(1, S+1), repeat=2)}
    
def max_power_cell(serial_number, s, S):
    '''Maximise the power for a square side `s`
    
    Returns the cell at the top-left of the `s` square
    - serial_number
    - s, the side legth
    - S, side length of the whole grid
    '''
    levels = power_levels(serial_number, S)
    def total_power(x, y):
        sq = its.product(range(x, x+s), range(y, y+s))
        return sum(levels[p] for p in sq)

    return max(
        (p for p in its.product(range(1,1+S-s), repeat=2)),
        key=lambda p: total_power(*p))

s, S = 3, 300
serial_number = 5468
assert max_power_cell(18, s, S) == (33, 45)
%time max_power_cell(5468, s, S)

Wall time: 419 ms


(243, 64)

In [37]:
def draw_cells(cells):
    'Draw the square grid of `cells`'
    S = int(math.sqrt(len(cells)))
    R = range(1, S+1)
    print('\n'.join(cat(f'{cells[(x,y)]:3}' for x in R) for y in R))

draw_cells(power_levels(6, 4))

 -4 -3 -3 -3
 -2 -2 -1 -1
 -1  0  0  1
  0  1  2  3


For the second part of the question, we want to maximise the power in a square region over all possible square side lengths `s`. We start by accumulating the power over the grid. 

In [38]:
def cumulative_power(serial_number, S):
    cells = power_levels(serial_number, S)
    R = range(1, S+1)
    # Accumulate rows top-to-bottom
    for y in R[1:]:
        cells.update({(x,y): cells[x,y]+cells[x,y-1] for x in R})

    # Accumulate columns left-to-right
    for x in R[1:]:
        cells.update({(x,y): cells[x,y]+cells[x-1,y] for y in R})
    
    return collections.Counter(cells)

draw_cells(power_levels(6,4))
print()
draw_cells(cumulative_power(6, 4))

 -4 -3 -3 -3
 -2 -2 -1 -1
 -1  0  0  1
  0  1  2  3

 -4 -7-10-13
 -6-11-15-19
 -7-12-16-19
 -7-11-13-13


Using the cumulative power grid we can read out the total power for a rectangular region.

In [39]:
def region_power(accu, t, l, b, r):
    '''Return the total power for a rectangular region
    
    accu - the cumulative powers over the grid
    tl - top-left corner
    br - bottom-right corner (included)
    '''
    return accu[(r,b)] - accu[(r,t-1)] - accu[(l-1,b)] + accu[(l-1,t-1)]

accu = cumulative_power(6, 4)
assert region_power(accu, 1, 1, 1, 1) == -4
assert region_power(accu, 1, 1, 3, 2) == -12

In [ ]:
def highest_powered_square(serial_number, S):
    R = range(S+1)
    accu = cumulative_power(serial_number, S)
    def square_power(lts):
        left, top, side = lts
        return region_power(accu, top, left, top+side-1, left+side-1)
    return max(
        ((left, top, side)
        for side in range(1, S+1)
        for top, left in its.product(range(1, S-side+2), repeat=2)),
        key=square_power)

# Check the supplied example answers
assert highest_powered_square(18, 300) == (90, 269, 16)
assert highest_powered_square(42, 300) == (232, 251, 12)

print(serial_number, S)
%time highest_powered_square(serial_number, S)

## [Day 12](https://adventofcode.com/2018/day/12)
This looks like a one dimensional [game of life](http://wordaligned.org/life).

In [ ]:
def parse_input(data):
    'Read initial state and rules'
    rules = {}
    for line in data.splitlines():
        if 'initial state' in line:
            state = line.split()[2]
        elif '=>' in line:
            src, _, dst = line.split()
            rules[src] = dst
    return state, rules

def trim(state, pos0):
    begin, end = state.find('#'), state.rfind('#')
    return state[begin:end+1], pos0-begin

def grow_plants(state, rules):
    pos0 = 0
    while True:
        state, pos0 = trim(state, pos0)
        yield state, pos0
        # Pad the state with enough blanks to apply the rules
        padded = '....' + state + '....'
        state = cat(rules.get(padded[p:p+5], '.') for p,c in enumerate(padded[2:-2]))
        pos0 += 2

example_setup = '''\
initial state: #..#.#..##......###...###

...## => #
..#.. => #
.#... => #
.#.#. => #
.#.## => #
.##.. => #
.#### => #
#.#.# => #
#.### => #
##.#. => #
##.## => #
###.. => #
###.# => #
####. => #'''

# Let's check the output against the example shown
state, rules = parse_input(example_setup)
print('\n'.join(s for s, _ in take(grow_plants(state, rules), 5)))

In [ ]:
def sum_of_plant_positions(setup, time):
    plants = grow_plants(*parse_input(setup))
    state, pos0 = next(its.islice(plants, time, None))
    return sum(i - pos0 for i, c in enumerate(state) if c == '#')

assert sum_of_plant_positions(example_setup, 20) == 325
sum_of_plant_positions(input(12), 20)

For part two, we're asked for the value returned by `sum_of_plant_positions()` when `time=50_000_000_000`. This is way too big to run directly. Let's see if the pattern repeats.

In [ ]:
def repeat(setup):
    '''Grow plants from initial setup until a state repeats
    
    Returns (state, time of repeat, pos0 at repeat, time first seen, pos0 first seen)
    '''
    seen = {}
    for t, (state, pos0) in enumerate(grow_plants(*parse_input(setup))):
        if state in seen:
            return (state, t, pos0, *seen[state])
        seen[state] = t, pos0

state, t_repeat, pos0_repeat, t_first_seen, pos0_first_seen = repeat(input(12))
print(t_repeat, pos0_repeat, t_first_seen, pos0_first_seen)

This tells us the state at time `192` is the same as the state at time `191`, shifted one to the left. Now we can figure out the state at `time=50_000_000_000`.

In [ ]:
T = 50_000_000_000
pos0 = -96 - T + 192
sum(i - pos0 for i, c in enumerate(state) if c == '#')